<a href="https://colab.research.google.com/github/nimab16/Music-Generation-LSTM-GAN/blob/main/Keras_LSTM_Music_gerenation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google drive
Note: before running any cell, please check that your Runtime has GPU!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Clone codes form github


In [ ]:
!git clone https://github.com/subpath/Keras_music_gereration

# install the libraries
Here you can see which libraries with which version is used

In [ ]:
!pip install tensorflow==1.14.0
!pip install tensorflow-gpu==1.14.0
!pip install mido==1.2.9
!pip install Keras==2.1.2
!pip install sklearn==0.0
!pip install numpy==1.17.3

# Music gereration firth Keras and TensorFlow backend

Plan was simple:
1. Read midi file, convert it to matrix of features
2. Create simple model with Keras and LSTM to learn the pattern
3. Use subsample of initial midi file as a input for model to generate pure art
4. Save prediction from model to midi file
.
.
.
5. PROFIT


In [ ]:
import sys

#!{sys.executable} -m pip install -q mido tqdm pandas tensorflow scikit-learn
# on my macOS I couldn't import keras properly, so I reinstall it with this command
# maybe you will need it too 🤷‍♂️
# !{sys.executable} -m  pip uninstall -y tensorflow keras tf-nightly keras-nightly

In [ ]:
import mido
import numpy as np
from mido import Message, MidiFile, MidiTrack
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import sequence

##### Read midi file
You can choose another music to generate another song based on that

In [ ]:
mid = MidiFile(
    "/content/Keras_music_gereration/Samples/Nintendo_-_Pokemon_Fire_Red_Route_1_Piano_Cover_Hard_Version.mid"
)
notes = []

##### Extract notes sequence
for the sake of simplicity I will use only 1 channel, it's like playining on piano with 1 hand 🙃

In [ ]:
notes = []
for msg in mid:
    if not msg.is_meta and msg.channel == 0 and msg.type == "note_on":
        data = msg.bytes()
        notes.append(data[1])

#### Apply min-max scalling

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(np.array(notes).reshape(-1, 1))
notes = list(scaler.transform(np.array(notes).reshape(-1, 1)))

#### Prepare features for training and data subsample for prediction

In [ ]:
# LSTM layers requires that data must have a certain shape
# create list of lists fist
notes = [list(note) for note in notes]

# subsample data for training and prediction
X = []
y = []
# number of notes in a batch
n_prev = 30
for i in range(len(notes) - n_prev):
    X.append(notes[i : i + n_prev])
    y.append(notes[i + n_prev])
# save a seed to do prediction later
X_test = X[-300:]
X = X[:-300]
y = y[:-300]

#### Made sequential model with several layers, use LSTM as it time dependent data

I also whant to save checkpoints

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.6))
model.add(Dense(1))
model.add(Activation("linear"))
optimizer = Adam(learning_rate=0.001)
model.compile(loss="mse", optimizer=optimizer)
filepath = "/content/Keras_music_gereration/Checkpoints/checkpoint_model_{epoch:02d}.hdf5"
model_save_callback = ModelCheckpoint(
    filepath,
    monitor="val_acc",
    verbose=1,
    save_best_only=False,
    mode="auto",
    save_freq=5,
)

#### Train your model.
It might take a while

In [ ]:
model.fit(np.array(X), np.array(y), 32, 10, verbose=1, callbacks=[model_save_callback])

#### Make a prediction

In [ ]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1, 1)))
prediction = [int(i) for i in prediction]

#### Save your result to new midi file

In [ ]:
mid = MidiFile()
track = MidiTrack()
t = 0
for note in prediction:
    msg_on = Message.from_dict({'type': 'note_on', 'channel': 0, 'note': note, 'velocity': 67, 'time':0})
    # you need to add some pauses "note_off"
    msg_off = Message.from_dict({'type': 'note_off', 'channel': 0, 'note': note, 'velocity': 67, 'time':64})
    track.append(msg_on)
    track.append(msg_off)
    track.append(msg_off)
mid.tracks.append(track)
mid.save("/content/drive/MyDrive/GAN music/LSTM_music_10_epoch.mid")

#### Just listen to it. The result is surreal!